# Démarche MLOps - Tracking, Versioning et CI/CD

Ce notebook présente la mise en œuvre de la démarche MLOps pour le projet Air Paradis : tracking des expérimentations, versioning des modèles, tests automatisés et déploiement continu.

## Principes MLOps

Le MLOps vise à industrialiser le cycle de vie des modèles ML :

- **Reproductibilité** : Garantir que les expérimentations peuvent être reproduites
- **Traçabilité** : Suivre toutes les versions de code, données et modèles
- **Automatisation** : Automatiser tests, déploiement et monitoring
- **Qualité** : Garantir la qualité via tests automatisés
- **Monitoring** : Surveiller les performances en production

```
DÉVELOPPEMENT          →    CI/CD PIPELINE       →    PRODUCTION
┌──────────────────┐       ┌──────────────────┐       ┌──────────────────┐
│ Notebooks        │       │ GitHub Actions   │       │ Heroku           │
│ MLFlow Tracking  │  →    │ Tests pytest     │  →    │ API FastAPI      │
│ Git/GitHub       │       │ Docker Build     │       │ PostHog Monitor  │
└──────────────────┘       └──────────────────┘       └──────────────────┘
```

In [1]:
# Import des bibliothèques
import mlflow
import mlflow.sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliothèques importées")

✓ Bibliothèques importées


## Tracking avec MLFlow

MLFlow permet de gérer le cycle de vie complet du ML :
- **Tracking** : Enregistrer paramètres, métriques et artefacts
- **Models** : Gérer et déployer les modèles
- **Model Registry** : Versionner les modèles en production

Configuration utilisée :

In [2]:
# Configuration de MLFlow
mlflow.set_tracking_uri("file:///home/thomas/mlruns")  # URI local (ou serveur distant)
mlflow.set_experiment("sentiment-analysis-twitter")

print(f"MLFlow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment actif: {mlflow.get_experiment_by_name('sentiment-analysis-twitter').name}")

MLFlow Tracking URI: file:///home/thomas/mlruns
Experiment actif: sentiment-analysis-twitter


### Exemple de tracking d'un run

Voici comment chaque expérimentation est trackée dans nos notebooks :

In [3]:
# Exemple de code utilisé dans les notebooks pour tracker un modèle
example_code = '''
with mlflow.start_run(run_name="logistic-regression-tfidf"):
    # Log des paramètres
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("preprocessing", "lemmatization")
    mlflow.log_param("vectorizer", "TfidfVectorizer")
    mlflow.log_param("max_features", 10000)
    mlflow.log_param("C", 1.0)
    
    # Entraînement du modèle
    model = LogisticRegression(C=1.0, max_iter=1000)
    model.fit(X_train_tfidf, y_train)
    
    # Prédictions et métriques
    y_pred = model.predict(X_test_tfidf)
    y_proba = model.predict_proba(X_test_tfidf)[:, 1]
    
    # Log des métriques
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba))
    mlflow.log_metric("precision", precision_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    
    # Log du modèle
    mlflow.sklearn.log_model(model, "model")
    
    # Log des artefacts (graphiques, matrices de confusion)
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.savefig('confusion_matrix.png')
    mlflow.log_artifact('confusion_matrix.png')
'''

print("Exemple de code de tracking MLFlow :")
print("="*80)
print(example_code)
print("="*80)

Exemple de code de tracking MLFlow :

with mlflow.start_run(run_name="logistic-regression-tfidf"):
    # Log des paramètres
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("preprocessing", "lemmatization")
    mlflow.log_param("vectorizer", "TfidfVectorizer")
    mlflow.log_param("max_features", 10000)
    mlflow.log_param("C", 1.0)
    
    # Entraînement du modèle
    model = LogisticRegression(C=1.0, max_iter=1000)
    model.fit(X_train_tfidf, y_train)
    
    # Prédictions et métriques
    y_pred = model.predict(X_test_tfidf)
    y_proba = model.predict_proba(X_test_tfidf)[:, 1]
    
    # Log des métriques
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba))
    mlflow.log_metric("precision", precision_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    
    # Log du modèle
    

### Visualisation dans MLFlow UI

Pour visualiser les expérimentations :

```bash
mlflow ui --backend-store-uri file:///home/thomas/mlruns
```

L'interface permet de comparer les runs, visualiser les métriques et télécharger les modèles.

In [4]:
# Récupération programmatique des runs MLFlow
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("sentiment-analysis-twitter")

if experiment:
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["metrics.f1_score DESC"],
        max_results=10
    )
    
    print("\n" + "="*100)
    print("RUNS MLFLOW ENREGISTRÉS (Top 10 par F1-Score)")
    print("="*100)
    
    runs_data = []
    for run in runs:
        runs_data.append({
            'Run ID': run.info.run_id[:8],
            'Model': run.data.params.get('model_type', 'N/A'),
            'Preprocessing': run.data.params.get('preprocessing', 'N/A'),
            'F1-Score': f"{run.data.metrics.get('f1_score', 0):.4f}",
            'Accuracy': f"{run.data.metrics.get('accuracy', 0):.4f}",
            'ROC-AUC': f"{run.data.metrics.get('roc_auc', 0):.4f}",
            'Date': run.info.start_time
        })
    
    df_runs = pd.DataFrame(runs_data)
    print(df_runs.to_string(index=False))
    print("="*100)
    print(f"\nTotal runs enregistrés : {len(runs)}")
else:
    print("⚠️ Experiment 'sentiment-analysis-twitter' non trouvé")
    print("   Les runs seront créés lors de l'exécution des notebooks de modélisation")


RUNS MLFLOW ENREGISTRÉS (Top 10 par F1-Score)
  Run ID Model Preprocessing F1-Score Accuracy ROC-AUC          Date
6ed6b866   N/A lemmatization   0.0000   0.0000  0.0000 1767290175909
bc350ec2   N/A Lemmatization   0.0000   0.0000  0.0000 1766977882182
7a7c580e   N/A Lemmatization   0.0000   0.0000  0.0000 1766972672796
f4e09554   N/A      Stemming   0.0000   0.0000  0.0000 1766967560870
54aecd1c   N/A Lemmatization   0.0000   0.0000  0.0000 1766962850056
51ba5d83   N/A Lemmatization   0.0000   0.0000  0.0000 1766914867879
83d935b3   N/A Lemmatization   0.0000   0.0000  0.0000 1766914184369
af9429a3   N/A Lemmatization   0.0000   0.0000  0.0000 1766913904592
17613af1   N/A Lemmatization   0.0000   0.0000  0.0000 1766896055309
b49831ba   N/A Lemmatization   0.0000   0.0000  0.0000 1766891485037

Total runs enregistrés : 10


Le tracking MLFlow permet de retrouver facilement toutes les expérimentations passées, comparer les performances et reproduire n'importe quel résultat.

## Model Registry

Le Model Registry gère le cycle de vie des modèles :
- **None** → En développement
- **Staging** → En validation
- **Production** → Déployé
- **Archived** → Déprécié

In [5]:
# Exemple d'enregistrement d'un modèle dans le registry
example_registry = '''
# Après avoir entraîné et logué un modèle dans un run
with mlflow.start_run(run_name="best-logistic-regression") as run:
    # ... entraînement et logging ...
    
    # Enregistrer le modèle dans le registry
    model_uri = f"runs:/{run.info.run_id}/model"
    mv = mlflow.register_model(model_uri, "sentiment-classifier")
    
    print(f"Modèle enregistré: {mv.name}, version {mv.version}")

# Promouvoir un modèle en Production
client = MlflowClient()
client.transition_model_version_stage(
    name="sentiment-classifier",
    version=1,
    stage="Production"
)
'''

print("Exemple d'enregistrement dans Model Registry :")
print("="*80)
print(example_registry)
print("="*80)

Exemple d'enregistrement dans Model Registry :

# Après avoir entraîné et logué un modèle dans un run
with mlflow.start_run(run_name="best-logistic-regression") as run:
    # ... entraînement et logging ...
    
    # Enregistrer le modèle dans le registry
    model_uri = f"runs:/{run.info.run_id}/model"
    mv = mlflow.register_model(model_uri, "sentiment-classifier")
    
    print(f"Modèle enregistré: {mv.name}, version {mv.version}")

# Promouvoir un modèle en Production
client = MlflowClient()
client.transition_model_version_stage(
    name="sentiment-classifier",
    version=1,
    stage="Production"
)



### 3.3 Chargement d'un Modèle depuis le Registry

In [6]:
# Exemple de chargement d'un modèle depuis le registry
example_load = '''
import mlflow.sklearn

# Charger la dernière version en Production
model_name = "sentiment-classifier"
model_version_uri = f"models:/{model_name}/Production"
model = mlflow.sklearn.load_model(model_version_uri)

# Utiliser le modèle pour prédire
prediction = model.predict(["This is a great product!"])
print(f"Prediction: {prediction}")
'''

print("Exemple de chargement depuis Model Registry :")
print("="*80)
print(example_load)
print("="*80)

Exemple de chargement depuis Model Registry :

import mlflow.sklearn

# Charger la dernière version en Production
model_name = "sentiment-classifier"
model_version_uri = f"models:/{model_name}/Production"
model = mlflow.sklearn.load_model(model_version_uri)

# Utiliser le modèle pour prédire
prediction = model.predict(["This is a great product!"])
print(f"Prediction: {prediction}")



## Versioning avec Git

Le projet est versionné sur GitHub avec la structure suivante :

```
openclassrooms-projet7/
├── api/                # Code de l'API FastAPI
├── streamlit/          # Interface utilisateur
├── notebooks/          # Expérimentations
├── livrables/          # Notebooks finaux
├── models/             # Modèles sérialisés
├── tests/              # Tests unitaires
├── requirements.txt    # Dépendances
└── README.md           # Documentation
```

### Historique des commits

### 4.2 Historique Git

Le projet contient au moins 3 versions distinctes accessibles via Git :

In [7]:
%%bash
# Afficher l'historique des commits
cd /mnt/c/Users/Thomas/Documents/Code/openclassrooms/openclassrooms-projet7
git log --oneline --graph --all -20

* 936761d test
* 4a066b5 test
* de4d81b Add demo tweets for presentation
* 7ccd9d9 Add final deliverables: blog article, presentation plan, and checklist
* 4e10814 Remove API keys from documentation - use empty strings
* b27bc26 Replace Azure Application Insights with PostHog Analytics
* a2e12bd Add Heroku API URL to Streamlit configuration
* c2c5669 Separate Streamlit interface into dedicated folder
* f1cfd00 Add requirements.txt at root for Streamlit Cloud
* 1fd5877 Fix Streamlit requirements versions for deployment
* 6b9fe0b improve api
* 5d40654 livrables
* c74dd68 bert 20 epoch
* 7cb1c3a Update confusion matrix and training history visualizations for BERT model
* 86a1313 notebook 5 fix
* 3658bd5 notebook 5
* 7c562b5 Clean up API directory
* 382766c Update scikit-learn to 1.7.2 to match vectorizer version
* ffde591 Add vectorizer idf_ attribute check on load
* 5afc9bf Force Heroku rebuild to load new vectorizer


### 4.3 Fichiers Clés de Versioning

#### requirements.txt

Liste de tous les packages avec versions exactes pour garantir la reproductibilité :

In [8]:
# Afficher le fichier requirements.txt
with open('../requirements.txt', 'r') as f:
    requirements = f.read()

print("Fichier requirements.txt :")
print("="*80)
print(requirements)
print("="*80)

Fichier requirements.txt :
# Streamlit Application Requirements

# Streamlit
streamlit>=1.28.0

# Data Processing
pandas>=2.0.0
numpy>=1.24.0,<2.0.0

# Visualization
plotly>=5.0.0

# API Calls
requests>=2.31.0

# Utilities
python-dotenv>=1.0.0

# Azure Application Insights (optionnel - pour monitoring)
opencensus-ext-azure>=1.1.0



#### .gitignore

Fichiers à ignorer (données volumineuses, secrets, cache) :

In [9]:
# Afficher le fichier .gitignore
import os
gitignore_path = '../.gitignore'

if os.path.exists(gitignore_path):
    with open(gitignore_path, 'r') as f:
        gitignore = f.read()
    
    print("Fichier .gitignore :")
    print("="*80)
    print(gitignore)
    print("="*80)
else:
    print("⚠️ Fichier .gitignore non trouvé")

Fichier .gitignore :
.venv/

data/

# Fichiers de modèles volumineux
models/**/*.h5
models/**/*.bin
*.h5


Le `.gitignore` exclut les fichiers volumineux (modèles, données) et sensibles (secrets).

## Tests unitaires

Les tests automatisés garantissent la qualité du code et évitent les régressions.

Exemple de tests de l'API :

In [10]:
# Exemple de fichier tests/test_api.py
test_api_code = '''
import pytest
import json
from api.app import app

@pytest.fixture
def client():
    """Créer un client de test Flask"""
    app.config['TESTING'] = True
    with app.test_client() as client:
        yield client

def test_health_check(client):
    """Test du endpoint racine"""
    response = client.get('/')
    assert response.status_code == 200
    assert response.json['status'] == 'ok'

def test_predict_positive(client):
    """Test de prédiction avec tweet positif"""
    data = {"text": "I love this airline! Great service!"}
    response = client.post('/predict', 
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 200
    result = response.json
    assert 'sentiment' in result
    assert 'proba' in result
    assert result['sentiment'] in [0, 1]
    assert 0 <= result['proba'] <= 1

def test_predict_negative(client):
    """Test de prédiction avec tweet négatif"""
    data = {"text": "Terrible experience! Never flying again!"}
    response = client.post('/predict',
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 200
    result = response.json
    assert result['sentiment'] == 0  # Négatif

def test_predict_missing_text(client):
    """Test avec texte manquant"""
    response = client.post('/predict',
                          data=json.dumps({}),
                          content_type='application/json')
    
    assert response.status_code == 400
    assert 'error' in response.json

def test_predict_empty_text(client):
    """Test avec texte vide"""
    data = {"text": ""}
    response = client.post('/predict',
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 400

def test_batch_predict(client):
    """Test de prédiction batch"""
    data = {
        "texts": [
            "Great flight!",
            "Terrible service!",
            "Average experience"
        ]
    }
    response = client.post('/batch_predict',
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 200
    result = response.json
    assert 'predictions' in result
    assert len(result['predictions']) == 3
'''

print("Exemple de tests unitaires (tests/test_api.py) :")
print("="*80)
print(test_api_code)
print("="*80)

Exemple de tests unitaires (tests/test_api.py) :

import pytest
import json
from api.app import app

@pytest.fixture
def client():
    """Créer un client de test Flask"""
    app.config['TESTING'] = True
    with app.test_client() as client:
        yield client

def test_health_check(client):
    """Test du endpoint racine"""
    response = client.get('/')
    assert response.status_code == 200
    assert response.json['status'] == 'ok'

def test_predict_positive(client):
    """Test de prédiction avec tweet positif"""
    data = {"text": "I love this airline! Great service!"}
    response = client.post('/predict', 
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 200
    result = response.json
    assert 'sentiment' in result
    assert 'proba' in result
    assert result['sentiment'] in [0, 1]
    assert 0 <= result['proba'] <= 1

def test_predict_negative(client):
    """Test de prédi

Exécution des tests :

```bash
pytest tests/ -v --cov=api
```

## Pipeline CI/CD

Le déploiement est automatisé via GitHub/Heroku :

1. **Push sur main** → Déclenche le pipeline
2. **Tests automatiques** → Validation pytest
3. **Build** → Création de l'environnement
4. **Deploy** → Mise à jour de l'API sur Heroku

Exemple de workflow GitHub Actions :

In [11]:
# Exemple de workflow GitHub Actions
workflow_yaml = '''
name: CI/CD Pipeline

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  test:
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v2
    
    - name: Set up Python
      uses: actions/setup-python@v2
      with:
        python-version: 3.10
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r api/requirements.txt
        pip install pytest pytest-cov
    
    - name: Run tests
      run: |
        pytest tests/ -v --cov=api
    
    - name: Upload coverage reports
      uses: codecov/codecov-action@v2
  
  build:
    needs: test
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v2
    
    - name: Build Docker image
      run: |
        docker build -t airparadis-sentiment-api:latest ./api
    
    - name: Login to Docker Hub
      uses: docker/login-action@v1
      with:
        username: ${{ secrets.DOCKER_USERNAME }}
        password: ${{ secrets.DOCKER_PASSWORD }}
    
    - name: Push Docker image
      run: |
        docker tag airparadis-sentiment-api:latest ${{ secrets.DOCKER_USERNAME }}/airparadis-sentiment-api:latest
        docker push ${{ secrets.DOCKER_USERNAME }}/airparadis-sentiment-api:latest
  
  deploy:
    needs: build
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    
    steps:
    - name: Deploy to Azure Web App
      uses: azure/webapps-deploy@v2
      with:
        app-name: airparadis-sentiment-api
        publish-profile: ${{ secrets.AZURE_WEBAPP_PUBLISH_PROFILE }}
        images: ${{ secrets.DOCKER_USERNAME }}/airparadis-sentiment-api:latest
'''

print("Fichier .github/workflows/deploy.yml :")
print("="*80)
print(workflow_yaml)
print("="*80)

Fichier .github/workflows/deploy.yml :

name: CI/CD Pipeline

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  test:
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v2
    
    - name: Set up Python
      uses: actions/setup-python@v2
      with:
        python-version: 3.10
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r api/requirements.txt
        pip install pytest pytest-cov
    
    - name: Run tests
      run: |
        pytest tests/ -v --cov=api
    
    - name: Upload coverage reports
      uses: codecov/codecov-action@v2
  
  build:
    needs: test
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v2
    
    - name: Build Docker image
      run: |
        docker build -t airparadis-sentiment-api:latest ./api
    
    - name: Login to Docker Hub
      uses: docker/login-action@v1
      with:
        username: ${{ secrets.DOCKER_

Le pipeline CI/CD permet de déployer en quelques minutes au lieu de plusieurs heures, avec la garantie que tous les tests passent avant mise en production.

## Monitoring en production

Le monitoring via PostHog permet de suivre :
- Volume de requêtes
- Temps de réponse
- Taux d'erreur
- Feedbacks utilisateurs (prédictions corrigées)

Configuration dans l'API :

## 7. Monitoring en Production <a id="7-monitoring"></a>

### 7.1 Azure Application Insights

**Application Insights** permet de monitorer :
- Volume de requêtes
- Temps de réponse
- Taux d'erreur
- Traces personnalisées (prédictions non conformes)
- Exceptions et logs

### 7.2 Configuration dans l'API

In [12]:
# Exemple de configuration Application Insights dans Flask
app_insights_code = '''
from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.flask.flask_middleware import FlaskMiddleware
from opencensus.ext.azure.trace_exporter import AzureExporter
import logging

# Configuration du logger
logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(
    connection_string='InstrumentationKey=YOUR_KEY'
))

# Ajouter le middleware Flask
middleware = FlaskMiddleware(
    app,
    exporter=AzureExporter(
        connection_string='InstrumentationKey=YOUR_KEY'
    ),
    sampler=ProbabilitySampler(rate=1.0)
)

# Dans le endpoint de prédiction
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data.get('text')
    
    # Prédiction
    prediction = model.predict([text])[0]
    proba = model.predict_proba([text])[0][prediction]
    
    # Logger la prédiction
    logger.info(f"Prediction made: {prediction}, confidence: {proba:.2f}")
    
    # Si prédiction non conforme (faible confiance)
    if proba < 0.6:
        logger.warning(
            f"Low confidence prediction: {proba:.2f} for text: {text[:50]}",
            extra={'custom_dimensions': {
                'prediction': int(prediction),
                'confidence': float(proba),
                'text_length': len(text)
            }}
        )
    
    return jsonify({
        'sentiment': int(prediction),
        'proba': float(proba)
    })
'''

print("Exemple de configuration Application Insights :")
print("="*80)
print(app_insights_code)
print("="*80)

Exemple de configuration Application Insights :

from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.flask.flask_middleware import FlaskMiddleware
from opencensus.ext.azure.trace_exporter import AzureExporter
import logging

# Configuration du logger
logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(
    connection_string='InstrumentationKey=YOUR_KEY'
))

# Ajouter le middleware Flask
middleware = FlaskMiddleware(
    app,
    exporter=AzureExporter(
        connection_string='InstrumentationKey=YOUR_KEY'
    ),
    sampler=ProbabilitySampler(rate=1.0)
)

# Dans le endpoint de prédiction
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data.get('text')
    
    # Prédiction
    prediction = model.predict([text])[0]
    proba = model.predict_proba([text])[0][prediction]
    
    # Logger la prédiction
    logger.info(f"Prediction made: {prediction}, confidence: {proba:.2f}")
    
    # Si 

### Alertes configurées

Des alertes sont mises en place pour détecter les problèmes :
- **Latence > 500ms** pendant 5 minutes → Email équipe
- **Taux d'erreur > 5%** pendant 10 minutes → SMS
- **Volume anormal** (possible downtime) → Alerte immédiate

### Stratégie d'amélioration continue

**Indicateurs surveillés** :
- Taux de prédictions corrigées par les utilisateurs
- Distribution des sentiments (détection de drift)
- Nouveaux mots/hashtags non vus à l'entraînement

**Déclencheurs de re-training** :
- Plus de 1000 corrections collectées
- Taux de prédictions incorrectes > 25%
- Changement significatif dans la distribution des données

**Workflow de re-training** :
1. Collecte des feedbacks utilisateurs
2. Détection de drift via monitoring
3. Re-entraînement avec nouvelles données
4. Validation sur test set
5. A/B testing en staging
6. Promotion en production si amélioration

## Conclusion

Ce projet met en œuvre une démarche MLOps complète :

- **Tracking** : MLFlow pour suivre toutes les expérimentations
- **Versioning** : Git/GitHub + MLFlow Model Registry
- **Tests** : pytest avec couverture du code
- **CI/CD** : Déploiement automatique sur Heroku
- **Monitoring** : PostHog pour la surveillance en production

Cette infrastructure permet de déployer rapidement, de détecter les problèmes et d'améliorer le modèle en continu grâce aux feedbacks utilisateurs.